<a href="https://colab.research.google.com/github/PETEROA/Model-Compression-Transfer_Learning/blob/main/Dynamic_Compress.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install openai

In [2]:
!pip install tiktoken

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 13.3 MB/s eta 0:00:00


In [3]:
!python -m nltk.downloader punkt_tab

<frozen runpy>:128: RuntimeWarning: 'nltk.downloader' found in sys.modules after import of package 'nltk', but prior to execution of 'nltk.downloader'; this may result in unpredictable behaviour
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [4]:
!pip install rouge-score

  Preparing metadata (setup.py) ... done
  Created wheel for rouge-score: filename=rouge_score-0.1.2-py3-none-any.whl size=24935 sha256=539f8322ce5b18c902b2516ad1d257450fec323b6f0f759b0f405e8e95ce205e
  Stored in directory: /root/.cache/pip/wheels/1e/19/43/8a442dc83660ca25e163e1bd1f89919284ab0d0c1475475148
Successfully built rouge-score


In [5]:
!pip install scikit-learn

In [6]:
!pip install sentence-transformers

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 90.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 66.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 45.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 48.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [7]:
import openai
import tiktoken
import nltk
from collections import Counter
from heapq import nlargest
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from rouge_score import rouge_scorer




In [8]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

Utilize reinforcement learning principles to select and score sentences for text compression. Initialize a MinMaxscaler for normalizing scores. score_sentences method assigns scores to sentences based on keyword frequency.

In [9]:
class ReinforcementLearningCompressor:
  def __init__(self):
    self.scaler = MinMaxScaler()
    self.rewards = {}

  def score_sentences(self, sentences, keyword_freq, preserve_keywords):
    sentence_scores = {}
    for sent in sentences:
      words_in_sent = word_tokenize(sent)
      score = sum(keyword_freq.get(word.lower(), 0) for word in words_in_sent)
      if preserve_keywords:
        score += len(words_in_sent)
      sentence_scores[sent] = score

    # Normalize scores
    # The original line was: scores = np.array(list(sentence_scores.keys()))
    # This creates an array of sentences (strings), not scores (numbers)
    # The following line fixes this by creating an array of the scores:
    scores = np.array(list(sentence_scores.values())).reshape(-1, 1)  # Reshape for MinMaxScaler
    normalized_scores = self.scaler.fit_transform(scores).flatten()

    for i, sent in enumerate(sentence_scores.keys()):
      sentence_scores[sent] = normalized_scores[i]
      self.rewards[sent] = self.rewards.get(sent, 0) + normalized_scores[i]

    return sentence_scores

  def select_sentences(self, sentence_scores, compression_ratio):
      num_sentences_to_keep = max(1, int(len(sentence_scores) * compression_ratio))
      top_sentences = nlargest(num_sentences_to_keep, sentence_scores, key=sentence_scores.get)
      return sorted(top_sentences, key=lambda x: list(sentence_scores.keys()).index(x))

this function attempts to assess the quality of text compression using semantic similarity and Rogue scores (ROGUE is a metric for evaluating text summarization quality by measuring the overlap between generated and reference text). It measures word and phrase overlap between the original and compressed text. sentence-transformers model generates computes cosine similarity and generates embeddings to boost semantic preservation.


In [10]:
from rouge_score import rouge_scorer
from sentence_transformers import SentenceTransformer, util



def evaluate_compression(original_prompt, compressed_text):
  #evaluates the quality of compression using ROGUE and sematic similarity
  scorer = rouge_scorer.RougeScorer(['rouge1', 'rouge2', 'rougeL'], use_stemmer=True)
  scores = scorer.score(original_prompt, compressed_text)

  model = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')
  embeddings_original = model.encode(original_prompt, convert_to_tensor=True)
  embeddings_compressed = model.encode(compressed_text, convert_to_tensor=True)
  similarity = util.pytorch_cos_sim(embeddings_original, embeddings_compressed).item()

  return{
      'rouge1': scores['rouge1'].fmeasure,
      'rouge2': scores['rouge2'].fmeasure,
      'rougeL': scores['rougeL'].fmeasure,
      'Semantic similarity': similarity
  }


the compress_prompt function tokenizes the input into words and sentences, calculates keyword frequency and removes stopwords. the rl_compressor scores the sentences based on keyword importance and selects the most relevant sentences according to a specified compression ratio.

In [11]:
def compress_prompt (prompt: str, compression_ratio: float = 0.5, model: str = "gpt-4-turbo", preserve_keywords: bool = True) -> str:
  #Tokenizer for OpenAI models
  enc = tiktoken.get_encoding("cl100k_base")

  #Tokenize sentences
  sentences = sent_tokenize(prompt)

  #Extract words using NLTK
  words = word_tokenize(prompt)
  words = [word.lower() for word in words if word.isalpha()]
  stop_words = set(stopwords.words('english'))
  filterd_words = [word for word in words if not word in stop_words]
  keyword_freq = Counter(filterd_words)

  #Reinforcement Learning Compressor
  rl_compressor = ReinforcementLearningCompressor()

  #Score sentences
  sentence_scores = rl_compressor.score_sentences(sentences, keyword_freq, preserve_keywords)
  top_sentences = rl_compressor.select_sentences(sentence_scores, compression_ratio)


  return " ".join(top_sentences)


In [17]:
#example usage
prompt_text = """
Artificial intelligence is transforming various industries, including healthcare, finance, and education.
Recent advancements in deep learning have enabled machines to perform tasks that were previously thought to be exclusive to humans.
However, ethical concerns and potential biases in AI models remain significant challenges.
Researchers are actively working on developing fair and interpretable AI systems to mitigate these issues.
AI is also being integrated into creative fields, such as music and writing, leading to discussions about authorship and originality.
"""
compressed_text = compress_prompt(prompt_text, 0.5, preserve_keywords=True)


In [18]:
evaluation_results = evaluate_compression(prompt_text, compressed_text)
print(evaluation_results)


{'rouge1': 0.6779661016949152, 'rouge2': 0.6551724137931034, 'rougeL': 0.6779661016949152, 'Semantic similarity': 0.7822194695472717}
